# Maximum uncertainty calculation

In [6]:
from pathlib import Path
from yaml import safe_load

import numpy as np

from tabulate import tabulate
from IPython.display import HTML, display

from scipy.spatial.transform import Rotation as R

from adaptive_planner.executor.orthomosaic import OrthomosaicSimConfig

Some helper functions:

In [2]:
def _display_table(table_html: str, title: int | None = None) -> None:
    display(
        HTML(
            f"""
        {"<h3>" + str(title) + "</h3>" if title else ""}
        <div style="max-height: 300px; overflow-y: auto; border: 1px solid #ccc;">
            {table_html}
        </div>
        """
        )
    )

Create a function to access the maximum uncertainty:

In [3]:
def create_config(config_file: Path) -> OrthomosaicSimConfig:
    with config_file.open("r") as config_file_handler:
        data = safe_load(config_file_handler)
        return OrthomosaicSimConfig(**data)


def get_max_uncertainty(config_file: Path, altitude: float) -> float:
    config = create_config(config_file)

    rotation_matrix = R.from_euler(
        "xyz", [config.roll_uncertainty_deg, config.pitch_uncertainty_deg, config.heading_uncertainty_deg], degrees=True
    ).as_matrix()
    position_uncertainty = rotation_matrix @ np.array([0, 0, altitude + config.altitude_uncertainty_m])
    position_uncertainty += config.position_uncertainty_m
    return position_uncertainty[:2].max()

Define the levels:

In [4]:
uncertainty_config_files = {
    "perfect": Path("experiments/localization_uncertainty/perfect.yaml"),
    "good": Path("experiments/localization_uncertainty/good.yaml"),
    "decent": Path("experiments/localization_uncertainty/decent.yaml"),
    "poor": Path("experiments/localization_uncertainty/poor.yaml"),
    "very poor": Path("experiments/localization_uncertainty/very_poor.yaml"),
}

assert all(f.is_file() for f in uncertainty_config_files.values())

In [10]:
altitudes = [12, 24, 36, 48]

rows = []
for k, v in uncertainty_config_files.items():
    rows.append([k] + [get_max_uncertainty(v, alt) for alt in altitudes])

_display_table(
    tabulate(rows, headers=["Uncertainty level"] + [f"{alt}m" for alt in altitudes], tablefmt="html", floatfmt=".3f"),
    title="Max position error for each uncertainty level",
)

Uncertainty level,12m,24m,36m,48m
perfect,0.000,0.000,0.000,0.000
good,0.121,0.226,0.332,0.438
decent,0.244,0.457,0.670,0.883
poor,0.537,0.969,1.402,1.835
very poor,1.348,2.477,3.606,4.735
